In [41]:
import pandas as pd # data frames
import numpy as np

In [42]:
# read data
df = pd.read_csv("https://raw.githubusercontent.com/askoshiyama/mli-cohort3/master/boston.csv")
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,T1
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


Repeated K Fold CV (5 outer, 5 inner, repeat 5 times):
RepeatedKFold repeats K-Fold n times. It can be used when one requires to run KFold n times, producing different splits in each repetition.
https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators

In [43]:
# defining vars
input_vars = ["V1", "V2", "V3", "V4", "V5", "V6", "V7",
              "V8", "V9", "V10", "V11", "V12", "V13"]
target = ["T1"]

In [44]:
X = df.drop(labels=target, axis=1)
X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33


In [45]:
y = df.drop(labels=input_vars, axis=1)
y.head()

,T1
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


In [46]:
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from interpret.glassbox import ExplainableBoostingRegressor 
from xgboost import XGBRegressor
from sklearn.svm import SVR

# define the models for selection along with their hyperparameters
# we try to find meaningful ranges for each hyperparameter. Among the models chosen for selection,
# we also include an MLP neural network as it presents the most recent advancement in supervised learning domain.
models_and_parameters = {
    'dummy': (DummyRegressor(), {}),
    'linReg': (LinearRegression(), {}),
    'ridge': (Ridge(), 
              {'alpha': np.linspace(0.00001, 0.8, num=50)}),
    'svr': (SVR(kernel='rbf'),
              {'C': [0.01, 0.05, 0.1, 0.5, 1], 
               'gamma': [0.001, 0.0001, 0.00001]}),
    'krr': (KernelRidge(), 
              {'kernel': ["poly","rbf"],
               'degree': [2,3,4],
               'alpha': np.linspace(0.00001, 0.8, num=50)}),
    'dtr': (DecisionTreeRegressor(),
              {"min_samples_split": [2, 10, 20, 40], 
               "max_depth": [2, 6, 8],
               "min_samples_leaf": [1, 20, 40, 100],
               "max_leaf_nodes": [None, 5, 20, 100]}),
    'rf': (RandomForestRegressor(n_estimators=100),
              {'max_depth': [10, 50, 100, 150, 200]}),
    'gbr': (GradientBoostingRegressor(),
              {"max_depth": [2, 6, 8],
               "learning_rate": [0.1, 0.01, 0.001],
               "n_estimators": [50, 100, 150]}),
    'ebr': (ExplainableBoostingRegressor(), {}),
    'xgbr': (XGBRegressor(), 
              {"max_depth": [2, 6, 8],
               "learning_rate": [0.1, 0.01, 0.001],
               "n_estimators": [50, 100, 150]}),
    'mlp': (MLPRegressor(early_stopping=True),
                {'hidden_layer_sizes': [(64,),(128,)],
                 'activation': ['relu'],
                 'solver':['lbfgs', 'adam'], 
                 'alpha':[0.0001],
                 'batch_size':['auto'], 
                 'learning_rate':['adaptive', 'constant'],
                 'learning_rate_init':[0.1, 0.01, 0.001], 
                 'max_iter':[1600]})
}

In [47]:
from sklearn import metrics

# performance metrics
perf_metrics = {"MAE": metrics.mean_absolute_error, 
                "EVS": metrics.explained_variance_score, 
                "MSE": metrics.mean_squared_error,
                "MDAE": metrics.median_absolute_error, 
                "R2": metrics.r2_score
               }
metric_greater_is_better = \
               {"MAE": False, 
                "EVS": True, 
                "MSE": False,
                "MDAE": False, 
                "R2": True
               }
scorer_metrics = {}
for pf in perf_metrics:
    scorer_metrics[pf] = metrics.make_scorer(score_func=perf_metrics[pf], greater_is_better=metric_greater_is_better[pf])

In [48]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold, KFold, cross_val_score, GridSearchCV
import numpy as np
from sklearn import metrics
from time import time


#######################################
# configuration
REPEAT = 1
SEED = None  # will ensure different results on each iteration
K_FOLD_NUM = 5
REFIT_METRIC = 'MSE'
#######################################

# use nested cross-validation for grid search and validation
# creates 5 folds for estimating generalization error
outer_folds = RepeatedKFold(n_repeats=REPEAT, n_splits=K_FOLD_NUM, random_state=SEED)

# when we train on a certain fold, we use a second cross-validation
# split in order to choose hyperparameters
inner_folds = KFold(n_splits=K_FOLD_NUM, random_state=SEED, shuffle=True)

i, z, innercv_results, inner_best_model = 0, 0, {}, {}
df_outer_results = pd.DataFrame(index=[0], columns=["Model", "Time"] + list(perf_metrics.keys()))

X_arr, y_arr = X.values, y.values

# outer loop
for (out_train, out_test) in outer_folds.split(X_arr, y_arr):
    print(f"Repetition {int(i / K_FOLD_NUM) + 1} Fold {i % K_FOLD_NUM + 1}")
    i += 1  # iteration
    # separation: train, test
    Xtrain, Xtest = X_arr[out_train], X_arr[out_test]
    ytrain, y_test = y_arr[out_train], y_arr[out_test]
    
    # inner loop -- all models
    fold_id = "fold_" + str(i)
    innercv_results[fold_id] = {}
    inner_best_model[fold_id] = {}
    start, end = [], []
    
    for model_name, (model, params) in models_and_parameters.items():
        start.append(time())
        print("\t", model_name)
        innercv_results[fold_id][model_name] = GridSearchCV(estimator=model, 
                                                            param_grid=params, 
                                                            scoring=scorer_metrics, 
                                                            iid=False,
                                                            cv=inner_folds, 
                                                            refit=REFIT_METRIC)    
        innercv_results[fold_id][model_name].fit(Xtrain, ytrain.ravel())
        inner_best_model[fold_id][model_name] = innercv_results[fold_id][model_name].best_estimator_
        end.append(time())        
    
    # outer loop prediction    
    for v, (model_name, (model, _)) in enumerate(models_and_parameters.items()):
        # prediction
        y_test_pred = inner_best_model[fold_id][model_name].predict(Xtest)
        df_outer_results.loc[z, "Model"] = model_name
        df_outer_results.loc[z, "Time"] = end[v] - start[v]
        
        # compute performance metrics
        for pf in perf_metrics.keys():
            df_outer_results.loc[z, pf] = perf_metrics[pf](y_test, y_test_pred)
        z += 1        

# final organisation
df_outer_results[list(perf_metrics.keys()) + ["Time"]] = df_outer_results[list(perf_metrics.keys()) + ["Time"]].astype(float)
print("done.")

Repetition 1 Fold 1
	 dummy
	 linReg
	 ridge
	 svr
	 krr


C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anacon

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.76163e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=4.05831e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned ma

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anacon

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anacon

...

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=8.95995e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=7.46362e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=9.03437e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.18798e-17): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=8.86896e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\e

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.2105e-17): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=9.16062e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=8.83721e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.00668e-17): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=7.78962e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\en

	 dtr
	 rf
	 gbr
	 ebr
	 xgbr
[20:47:01] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:47:01] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:47:01] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:47:01] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:47:01] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:47:01] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:

[20:47:04] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:47:04] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:47:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:47:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:47:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:47:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:47:05] WARNI

[20:47:07] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:47:07] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:47:07] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:47:07] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:47:07] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:47:08] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:47:08] WARNI

In [49]:
df_outer_results.pivot_table(index=["Model"], aggfunc="mean")

,EVS,MAE,MDAE,MSE,R2,Time
Model,,,,,,
dtr,8.022282e-01,2.889756,2.064071,16.524655,0.799623,2.266010
dummy,4.440892e-17,6.652439,4.772780,84.580330,-0.005888,0.013166
ebr,8.423336e-01,2.511109,1.853582,13.560967,0.840982,32.023352
gbr,8.722460e-01,2.226667,1.625764,10.659874,0.871085,11.103400
krr,8.311063e-01,2.593267,1.914664,14.073842,0.829516,31.665361
linReg,7.241873e-01,3.418812,2.679545,24.066982,0.714791,0.014583
mlp,7.536626e-01,2.968872,2.012849,20.303841,0.751188,168.534495
rf,8.747427e-01,2.199625,1.591700,10.493297,0.873389,6.678581
ridge,7.238936e-01,3.417720,2.646197,24.097117,0.714562,0.474544


In [67]:
# pick the best model according to the refit metric
best_models_mean = df_outer_results.pivot_table(index=["Model"], aggfunc="mean")
best_model_refit_metric = best_models_mean[REFIT_METRIC].idxmin()
best_model_entry = best_models_mean[best_models_mean.index == best_model_refit_metric]
best_model_entry

,EVS,MAE,MDAE,MSE,R2,Time
Model,,,,,,
xgbr,0.886614,2.138204,1.501778,9.622726,0.885738,8.196388


In [74]:
best_model_name = best_model_entry.index.values[0]
best_model_name

'xgbr'

Using the MSE refit metric, XGBRegressor is the best model. Now we pick the model that achieved the lowest MSE score:

In [56]:
best_model_results = df_outer_results[df_outer_results['Model'] == best_model_refit_metric]
best_model_results

,Model,Time,MAE,EVS,MSE,MDAE,R2
9,xgbr,8.509229,2.547968,0.883030,13.252438,1.950455,0.880030
20,xgbr,8.127516,2.162022,0.876657,9.441758,1.559738,0.876655
31,xgbr,8.306774,1.777350,0.925106,5.815094,1.201061,0.923743
42,xgbr,7.931075,2.094111,0.888624,9.161777,1.376532,0.888624
53,xgbr,8.107348,2.109571,0.859654,10.442563,1.421105,0.859639


In [78]:
idxmin = best_model_results[REFIT_METRIC].idxmin()

fold_id = int(idxmin / len(models_and_parameters))
best_estimator = innercv_results['fold_' + str(fold_id)][best_model_name].best_estimator_
best_params = innercv_results['fold_' + str(fold_id)][best_model_name].best_params_

In [79]:
best_estimator

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [80]:
best_params

{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100}

Discussion/Arguments: 

